# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [287]:
# !pip install ultralytics
# !pip install opencv-python

In [288]:
from ultralytics import YOLO , solutions
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [289]:
model = YOLO('/content/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [290]:
# Prepare the video capture
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [291]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [292]:
print("Height:", height)
print("Width:", width)
print("FPS:", fps)

Height: 1080
Width: 1920
FPS: 29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [293]:
output_path = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [294]:
# Regions
top_exit = (1000, 50, 1300, 200)
bottom_exit = (600, 900, 900, 1000)
left_exit = (400, 300, 600, 550)
right_exit = (1250, 450, 1450, 750)

In [295]:
# cars count
top_count = 0
bottom_count = 0
left_count = 0
right_count = 0

In [296]:
# Saves cars ID on sets
top_exited_ids = set()
bottom_exited_ids = set()
left_exited_ids = set()
right_exited_ids = set()

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

#Doesn't work!

In [297]:
# region_points = [(1000,1300),(900, 600),(400,600),(1450, 1250)]

In [298]:
# video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))


In [299]:
# counter = solutions.ObjectCounter(
#     view_img=True,
#     reg_pts=region_points,
#     names=model.names,
#     draw_tracks=True,
#     line_thickness=2,
# )

In [300]:
# while cap.isOpened():
#         success, im0 = cap.read()
#         if not success:
#             print("Video frame is empty or video processing has been successfully completed.")
#             break
#         tracks = model.track(im0, persist=True, show=False)
#         im0 = counter.start_counting(im0, tracks)
#         video_writer.write(im0)
#         cv2_imshow(im0)

#Trying Diffrent solution




In [301]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO model on the frame
    results = model.track(frame, persist=True)

    for result in results:
        for boxes in result.boxes:

            # Get Object details
            bbox = boxes.xyxy[0].cpu().numpy()          #Bounding Box
            class_id = int(boxes.cls[0].cpu().numpy())  #Class ID
            conf = boxes.conf[0].cpu().numpy()          #Confidence
            id = int(boxes.id[0].cpu().numpy())         #Object ID

            x1, y1, x2, y2 = map(int, bbox)
            # Get the center of the Object
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

            # Draw BBox
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

            label = f'ID: {id} Conf: {conf:.2f} Class:{class_id}'
            # Add labels for each car
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

            # Count vehicles based on their exit region
            # Top Reigon
            if top_exit[0] < center_x < top_exit[2] and top_exit[1] < center_y < top_exit[3]:
                if id not in top_exited_ids: # if the id is not in the set +1
                    top_count += 1
                    top_exited_ids.add(id) # add id to the set
            # Bottom Reigon
            elif bottom_exit[0] < center_x < bottom_exit[2] and bottom_exit[1] < center_y < bottom_exit[3]:
                if id not in bottom_exited_ids: # if the id is not in the set +1
                    bottom_count += 1
                    bottom_exited_ids.add(id) # add id to the set
            # Left Reigon
            elif left_exit[0] < center_x < left_exit[2] and left_exit[1] < center_y < left_exit[3]:
                if id not in left_exited_ids: # if the id is not in the set +1
                    left_count += 1
                    left_exited_ids.add(id) # add id to the set
            # Right Reigon
            elif right_exit[0] < center_x < right_exit[2] and right_exit[1] < center_y < right_exit[3]:
                if id not in right_exited_ids: # if the id is not in the set +1
                    right_count += 1
                    right_exited_ids.add(id) # add id to the set




    # Display the counts on the video frame
    cv2.putText(frame, f'Top Exit: {top_count}', (25, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, f'Bottom Exit: {bottom_count}', (25, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, f'Left Exit: {left_count}', (25, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, f'Right Exit: {right_count}', (25, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Display my name
    cv2.putText(frame, 'Ahmed Bashmmakh', (300, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Creating Reigons in the frames
    #Top
    cv2.rectangle(frame, top_exit[:2], top_exit[2:], (0, 0, 255), 2)
    #Bottom
    cv2.rectangle(frame, bottom_exit[:2], bottom_exit[2:], (0, 0, 255), 2)
    #Left
    cv2.rectangle(frame, left_exit[:2], left_exit[2:], (0, 0, 255), 2)
    #Right
    cv2.rectangle(frame, right_exit[:2], right_exit[2:], (0, 0, 255), 2)

    out.write(frame)

    # Display each frame
    # cv2_imshow(frame)




0: 384x640 11 cars, 2389.7ms
Speed: 3.8ms preprocess, 2389.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 2389.5ms
Speed: 4.3ms preprocess, 2389.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2406.0ms
Speed: 4.5ms preprocess, 2406.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 2501.1ms
Speed: 3.1ms preprocess, 2501.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 3634.7ms
Speed: 3.7ms preprocess, 3634.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 2418.8ms
Speed: 4.7ms preprocess, 2418.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 2431.2ms
Speed: 3.8ms preprocess, 2431.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 2393.7ms
Speed: 3.8ms preprocess, 2393.7ms inference, 0.9ms postpro

## Save and Submit Your Work

Submit both the notebook and the output video

In [302]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()


Process taking too long

#Source:
* https://learnopencv.com/yolov8-object-tracking-and-counting-with-opencv/
* https://docs.ultralytics.com/guides/object-counting/
* https://github.com/orgs/ultralytics/discussions/6738
* https://github.com/arpitmaur07/Object-Detection-and-Object-Counting/blob/main/main.py
* https://github.com/roboflow/notebooks/blob/main/notebooks/how-to-track-and-count-vehicles-with-yolov8.ipynb